# Introduction to Pandas

`pandas` is an awesome library for csv file manipulations, and so much more!

## First, setup

In [1]:
import pandas as pd
import numpy as np

## Series

The basic `pandas` data structure is `Series`. It represents a one-dimensional collection of values (similar to python native `list` and `numpy` `array`)

In [2]:
s = pd.Series([1, 2, 3])

In [3]:
print(s)

0    1
1    2
2    3
dtype: int64


## Data Frame